In [1]:
import pandas as pd
import xlwings as xw
from pathlib import Path

In [2]:
# Obtenemos nombre de carpetas dentro de un directorio
def obtener_carpetas():
    directorio = Path('../excel')
    lista_carpetas = [fichero.name for fichero in directorio.iterdir() if fichero.is_dir()]
           
    return lista_carpetas

In [3]:
# Obtenemos nombre de archivos en un directorio
def obtener_archivos(lista_carpetas, indice):

    archivos = Path('..\\excel', lista_carpetas[indice])
    lista_archivo = [fichero.name for fichero in archivos.iterdir() if fichero.is_file()]

    return lista_archivo

llamamos a las funciones para conocer carpetas y archivos

In [4]:
carpetas = obtener_carpetas()
carpetas

['2011']

In [5]:
archivos = obtener_archivos(carpetas, indice = 0)
archivos

['PLANILLA VENTA FLORES VERTICAL dic 2011 (MONO)2.xlsx',
 'PLANILLA VENTA FLORES VERTICAL nov 2011(MONO)2.xls',
 'PLANILLA VENTA FLORES VERTICAL nov 2011(MONO)2.xlsx',
 'PLANILLA VENTA FLORES VERTICAL oct 2011(MONO)2.xls',
 'PLANILLA VENTA FLORES VERTICAL oct 2011(MONO)2.xlsx',
 'PLANILLA VENTA FLORES VERTICAL sep2011(MONO)2.xls']

In [6]:
# Seleccionamos nombre de carpeta y archivo a explorar
dirname = carpetas[0]
filename = archivos[5]
password = "0811"

# Abrimos libro de excel
wb = xw.Book(f'../excel/{dirname}/{filename}', password=password)

In [7]:
# Extraemos nombre de hojas del libro
for i, e in enumerate(wb.sheet_names):
    print(i, ':', e)

0 : PARA IMPRIMIR
1 : 11-09-2011
2 : 17-09-2011
3 : 18-09-2011
4 : 19-09-2011
5 : 20-09-2011
6 : 21-09-2011
7 : 22-09-2011
8 : 23-09-2011
9 : 24-09-2011
10 : 25-09-2011
11 : 28-09-2011
12 : 29-09-2011
13 : 30-09-2011
14 : especial 26-09-11 a 30-09-11
15 : MES SEPTIEMBRE
16 : DEFAULT
17 : PLANILLA FLORES
18 : GRAFICA


In [8]:
#sheet = wb.sheets[18]
#df_control = sheet.used_range.options(pd.DataFrame, header=False, index=False).value

In [9]:
#df_caja = df_control.iloc[14:35, 16:22]

In [10]:
#df_caja

### ETL

In [13]:
# Itineramos entre las hojas utiles
for i in range(1,14):
    numero_hoja = i
    sheet = wb.sheets[numero_hoja]

    # Generamos dtaframe
    df = sheet.used_range.options(pd.DataFrame, header=False, index=False).value
    df2 =df.copy()
    #df2.head()
    
    # Seccionamos dataframe
    df_stock = df2.iloc[1:10, 16:25]
    df_precio = df2.iloc[1:3, 5:13]
    df_caja = df2.iloc[14:35, 16:22]

    # Creamos Dataframe vacios
    if numero_hoja <= 1:
        movimientos_stock = pd.DataFrame(columns= ['fecha', 'descripcion', 'astromelias', 'lisiantus', 'montoneras', 'gladiolo', 'ramito', 'crisantemo', 'rosa', 'clavel'])
        precios = pd.DataFrame(columns= ['fecha', 'astromelias', 'lisiantus', 'montoneras', 'gladiolo', 'ramito', 'crisantemo', 'rosa', 'clavel'])
        flujo_caja = pd.DataFrame(columns= ['fecha', '$ SALDO INICIAL', '$ RETIROS', '$ GASTOS', '$ Vtas TOTAL', '$ SALDO', '$ SALDO REAL'])
    
    
    # Ingestamos Dataframe Precios
    dicc_precios = { 'fecha' : wb.sheet_names[numero_hoja]}
    df_precio.iloc[0] = df_precio.iloc[0].apply(lambda x: x.lower())
    for i in range(len(df_precio.columns)):
        #print(df_precio.iloc[0,i], ':', df_precio.iloc[1,i])
        dicc_precios[df_precio.iloc[0,i]] = df_precio.iloc[1,i]
    precios = pd.concat([precios, pd.DataFrame(dicc_precios, index= [0])], ignore_index= True)
    
    # Ingestamos Dataframe Stock
    dicc_stock = { 'fecha' : wb.sheet_names[numero_hoja]}
    df_stock.iat[0,0] = 'descripcion'
    df_stock.iloc[0, 1:] = df_stock.iloc[0, 1:].apply(lambda x: x.lower())
    lista_descripcion = list(df_stock[16])[1:]

    for descripcion in lista_descripcion:
        indice = df_stock.index[df_stock[16] == descripcion].values[0]-1
        #print(descripcion, ':', indice)
        for i in range(len(df_stock.columns)):
            #pass
            #print(df_stock.iloc[0, i], ':', df_stock.iloc[indice, i])
            dicc_stock[df_stock.iloc[0, i]] = df_stock.iloc[indice, i]
        movimientos_stock = pd.concat([movimientos_stock, pd.DataFrame(dicc_stock, index= [0])], ignore_index= True)
    
    # Ingestamos Dataframe Caja
    dicc_caja = { 'fecha' : wb.sheet_names[numero_hoja],
                    '$ SALDO INICIAL' : 0, 
                    '$ RETIROS' : 0, 
                    '$ GASTOS' : 0, 
                    '$ Vtas TOTAL' : 0, 
                    '$ SALDO' : 0, 
                    '$ SALDO REAL' : 0
                }
    df_caja.reset_index(drop= True, inplace= True)
    indice_saldo_inicial = df_caja.index[df_caja[16] == '$ SALDO INICIAL'].values[0]
    indice_ventas_total = df_caja.index[df_caja[16] == '$ Vtas TOTAL'].values[0]
    indice_saldo = df_caja.index[df_caja[16] == '$ SALDO'].values[0]
    indice_saldo_real = df_caja.index[df_caja[16] == '$ SALDO REAL'].values[0]
    indice_subtotal = df_caja.index[df_caja[19] == 'SUBTOTAL'].values[0]

    dicc_caja['$ SALDO INICIAL'] = df_caja.iloc[indice_saldo_inicial, 1]
    dicc_caja['$ RETIROS'] = round(df_caja.iloc[indice_subtotal, 4], 2)
    dicc_caja['$ GASTOS'] = round(df_caja.iloc[indice_subtotal, 5], 2)
    dicc_caja['$ Vtas TOTAL'] = round(df_caja.iloc[indice_ventas_total, 1], 2)
    dicc_caja['$ SALDO'] = round(df_caja.iloc[indice_saldo, 1], 2)
    dicc_caja['$ SALDO REAL'] = df_caja.iloc[indice_saldo_real, 1]

    flujo_caja = pd.concat([flujo_caja, pd.DataFrame(dicc_caja, index= [0])], ignore_index= True)

In [14]:
# Manejo de datos Nan
precios = precios.fillna(axis = 0, method= "ffill").fillna(0)
movimientos_stock = movimientos_stock.fillna(0)
flujo_caja = flujo_caja.fillna(0)

In [15]:
# Generamos CSV
precios.to_csv('../csv/2011/precios sep-2011', index= False)
movimientos_stock.to_csv('../csv/2011/movimientos_stock sep-2011', index= False)
flujo_caja.to_csv('../csv/2011/flujo_caja sep-2011', index= False)

In [16]:
# Leemos CSV
precios = pd.read_csv('../csv/2011/precios sep-2011')
movimientos_stock = pd.read_csv('../csv/2011/movimientos_stock sep-2011')
flujo_caja = pd.read_csv('../csv/2011/flujo_caja sep-2011')

In [17]:
precios.head()

,fecha,astromelias,lisiantus,montoneras,gladiolo,ramito,crisantemo,rosa,clavel
0,11-09-2011,0,0,3.0,4.0,3.0,3.5,5.0,1.69
1,17-09-2011,0,0,3.0,4.0,3.0,3.5,5.0,1.69
2,18-09-2011,0,0,3.0,4.0,3.0,3.5,5.0,1.69
3,19-09-2011,0,0,3.0,4.0,3.0,3.5,5.0,1.69
4,20-09-2011,0,0,3.0,4.0,3.0,3.5,5.0,1.69


In [22]:
movimientos_stock.head(10)

,fecha,descripcion,astromelias,lisiantus,montoneras,gladiolo,ramito,crisantemo,rosa,clavel
0,11-09-2011,SALDO INICIAL,0.0,0.0,0.0,3.0,0.0,44.0,34.0,362.0
1,11-09-2011,COMPRA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,11-09-2011,VENTA TOTAL,0.0,0.0,0.0,3.0,0.0,24.0,23.0,133.0
3,11-09-2011,SERVICIO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0
4,11-09-2011,ROTAS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
5,11-09-2011,SALDO,0.0,0.0,0.0,0.0,0.0,20.0,11.0,215.0
6,11-09-2011,SALDO REAL,0.0,0.0,0.0,0.0,0.0,20.0,11.0,181.0
7,11-09-2011,DIFERENCIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-34.0
8,17-09-2011,SALDO INICIAL,0.0,0.0,0.0,0.0,0.0,20.0,11.0,181.0
9,17-09-2011,COMPRA,0.0,0.0,90.0,48.0,0.0,216.0,700.0,2400.0


In [21]:
flujo_caja.describe()

,$ SALDO INICIAL,$ RETIROS,$ GASTOS,$ Vtas TOTAL,$ SALDO,$ SALDO REAL
count,13.000000,13.000000,13.000000,13.000000,13.000000,13.000000
mean,136.530769,906.153846,50.826923,1056.178462,235.728462,137.803846
std,89.910814,1203.110818,121.407860,1206.933546,509.608939,85.150769
min,0.000000,0.000000,0.000000,202.370000,-230.330000,0.000000
25%,77.750000,200.000000,0.000000,264.740000,74.840000,77.750000
50%,138.150000,550.000000,0.000000,573.830000,116.570000,132.300000
75%,219.750000,800.000000,2.500000,922.230000,219.770000,219.750000
max,279.600000,3890.000000,392.250000,3832.170000,1879.090000,279.600000
